# Project 2 - Ali Nehrani 

In [108]:
import os
import nltk
import string
#from nltk import 
import numpy as np
import re as regex
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt 
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn import metrics, svm, model_selection, feature_extraction, linear_model, tree, ensemble, neural_network


In [109]:
#stopwords=nltk.corpus.stopwords.words("english")

In [110]:
def cl_ts_evaluation(clss, k, data, target, method=None, test_ratio=0.2):
    metric_ac_score = []
    precision = []
    recall = []
    accuracy = []
    f1 = []
    method_score=[]
    if method == 'kfold':
        kfold = model_selection.KFold(n_splits=k)
        for ind_train,ind_test in kfold.split(data):
            clss.fit(data[ind_train], target[ind_train])  
            method_score.append(clss.score(data[ind_test], target[ind_test]))
            y_pred = clss.predict(data[ind_test]) #
            metric_ac_score.append(metrics.accuracy_score(target[ind_test], y_pred))           
            precision.append(precision_score(target[ind_test], y_pred, average=None, pos_label=1)) # pos_label=1, average=’binary’,
            recall.append(recall_score(target[ind_test], y_pred,  average=None, pos_label=1))
            accuracy.append(accuracy_score(target[ind_test], y_pred))
            f1.append(f1_score(target[ind_test], y_pred, average=None, pos_label=1))
            return [np.mean(metric_ac_score), np.mean(precision), np.mean(recall), np.mean(accuracy), np.mean(f1), np.mean(method_score)]
                                
    else:
        for I in range(k):
            x_train, x_test, y_train, y_test = model_selection.train_test_split(data,target,test_size=test_ratio)
            clss.fit(x_train,y_train)
            metric_ac_score.append(clss.score(x_test,y_test))
            
    return np.mean(metric_ac_score)

In [111]:
fileAddress=os.path.join(os.getcwd(), 'twitter_sentiment_corpus.csv')
print('Start to loading first data from this file: \n', fileAddress)
tweetsData = pd.read_csv(fileAddress)

Start to loading first data from this file: 
 /home/bayes/Academic/Sensoy/Project2/ml-twitter-sentiment-analysis-develop/twitter_sentiment_corpus.csv


In [112]:
# preprocessing
# removing improper data
tweetsData = tweetsData.drop(tweetsData[tweetsData['Sentiment'] == 'irrelevant'].index)

df = tweetsData

df = df[df["TweetText"] != "Not Available"]
#removing numbers
r_num = regex.compile(r"\s?[0-9]+\.?[0-9]*")

text = df.loc[:, "TweetText"]

text.replace(r_num, "", inplace=True)

# removing other characters
for remove in map(lambda r: regex.compile(regex.escape(r)), [",", ":", "\"", "=", "&", ";", "%", "$",
                                                                     "@", "%", "^", "*", "(", ")", "{", "}",
                                                                     "[", "]", "|", "/", "\\", ">", "<", "-",
                                                                     "!", "?", ".", "'",
                                                                     "--", "---", "#"]):
            df.loc[:, "TweetText"].replace(remove, "", inplace=True)     

        
        
    
tweetsData = df        

In [113]:
# we can define manual stop words also
stopwords=nltk.corpus.stopwords.words("english")

In [116]:
from nltk.stem.snowball import SnowballStemmer
df = tweetsData['TweetText']

#sbEng = SnowballStemmer('english')
#for i in range(5113):
#    df[i] = ' '.join([sbEng.stem(item) for item in (df[i]).split(' ')])
#    df[i] = ' '.join([w for w in df[i] if w.lower() not in string.punctuation])
#    df[i] = ' '.join([w for w in df[i] if w.lower() not in stopwords])
    


    
#from nltk.tokenize import word_tokenize
#engTokens = [word_tokenize(text) for text in df]


#for i in range(5113):
#engTokens = ' '.join([w for w in engTokens if w.lower() not in string.punctuation])
#    engTokens[i] = ' '.join([w for w in engTokens[i] if w.lower() not in stopwords])
#    engTokens[i] = '\n'.join(regex.sub(r'(?:(?:http|https):\/\/)?([-a-zA-Z0-9.]{2,256}\.[a-z]{2,4})\b(?:\/[-a-zA-Z0-9@:%_\+.~#?&//=]*)?',"",engTokens[i],flags=regex.MULTILINE))
#    engTokens[i] = ' '.join( engTokens[i].split())
    
tweetsData['TweetText'] = df    




In [ ]:
df

In [28]:
df = tweetsData
# Tokinazition 
# first using the stemmer:
stemmer=nltk.PorterStemmer()

df["TweetText"] = list(map(lambda str: stemmer.stem(str.lower()), df["TweetText"]))

tweetsData = df

In [29]:
#data deistribution
import seaborn as sns
sns.set(color_codes=True)

df = tweetsData
negetive = len(df[df["Sentiment"] == "negative"])
positive = len(df[df["Sentiment"] == "positive"])
neutral = len(df[df["Sentiment"] == "neutral"])

t_size = negetive + positive + neutral

dist_negetive = negetive/t_size
dist_positive = positive/t_size
dist_neutral = neutral/t_size


In [30]:
# changin target to number classes
row_index = tweetsData[tweetsData['Sentiment'] == 'positive'].index
tweetsData.loc[row_index, 'Sentiment'] = 1 
row_index = tweetsData[tweetsData['Sentiment'] == 'neutral'].index
tweetsData.loc[row_index, 'Sentiment'] = 0 
row_index = tweetsData[tweetsData['Sentiment'] == 'negative'].index
tweetsData.loc[row_index, 'Sentiment'] = -1

In [31]:
#  tweet Target 
tweetData = np.array(tweetsData['TweetText'])
tweetTarget = np.array(tweetsData['Sentiment'].values, dtype="|S6")

In [33]:
# Vectorizer
# 0    TfidfVectorizer
tfid=feature_extraction.text.TfidfVectorizer(use_idf=True,sublinear_tf=True, stop_words=stopwords) #'english'
data_1=tfid.fit_transform(tweetData)

In [34]:
# bigram TfidfVectorizer
tfid2=feature_extraction.text.TfidfVectorizer(use_idf=True, sublinear_tf=True, ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1, stop_words=stopwords)#'english'
data_2=tfid2.fit_transform(tweetData)

In [35]:
# CountVectorizer
countVectorize = feature_extraction.text.CountVectorizer(stop_words=stopwords) #'english'
data_3=countVectorize.fit_transform(tweetData)

In [36]:
#  bigram count vectorizer
bigram_vectorizer = feature_extraction.text.CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1, stop_words=stopwords)#'english'
data_4 = bigram_vectorizer.fit_transform(tweetData)

In [37]:
#  HashingVectorizer
hashingVectorizer = feature_extraction.text.HashingVectorizer(n_features=100)
data_5 = hashingVectorizer.fit_transform(tweetData)

In [38]:
# parameter setting
maxAccuracy=0

In [39]:
# all prints
models = [linear_model.LogisticRegression(),
          svm.LinearSVC(),
          tree.DecisionTreeClassifier(max_depth=10),
          ensemble.RandomForestClassifier(),
          BernoulliNB()
         ]
data = [data_1,
       data_2,
       data_3,
       data_4,
       data_5]
data_ind = ['data_1',
       'data_2',
       'data_3',
       'data_4',
       'data_5']
method=[None, 'kfold']
    

In [62]:

for meth in method:
    
    for model in models:
        i=0
        for dat in data:
            print("Model: {}===>".format(str(type(model).__name__), data_ind[i], meth))
            print("--------------------------------")
            print("Data: {}".format(data_ind[i]))
            print("--------------------------------")
            print("Method: {}".format(meth))
            print("--------------------------------")
            accuracy = cl_ts_evaluation(clss=model, k=10, data=dat, target=tweetTarget, method=meth) 
            if meth == 'kfold':
                print("          ----- Results -------              ")
                print("                                             ")
                print("metric_ac_score: " + str(accuracy[0]))
                print("Precision: " + str(accuracy[1]))
                print("Accuracy:  " + str(accuracy[3]))
                print("F1: " + str(accuracy[4]))
                print("Method Score: " + str(accuracy[5]))
                print("Recall: " + str(accuracy[2]))
                print("===============================================")
            else:
                print('Accuracy of the method: \t\t{0:.4f}'.format(accuracy))
                print("************************************************************")
            i+=1   
    #accuracy = eval(cls=svm.LinearSVC(), k=10, data=newData, target=tweetTarget)

Model: LogisticRegression===>
--------------------------------
Data: data_1
--------------------------------
Method: None
--------------------------------
Accuracy of the method: 		0.7153
************************************************************
Model: LogisticRegression===>
--------------------------------
Data: data_2
--------------------------------
Method: None
--------------------------------
Accuracy of the method: 		0.6909
************************************************************
Model: LogisticRegression===>
--------------------------------
Data: data_3
--------------------------------
Method: None
--------------------------------
Accuracy of the method: 		0.7601
************************************************************
Model: LogisticRegression===>
--------------------------------
Data: data_4
--------------------------------
Method: None
--------------------------------
Accuracy of the method: 		0.7628
************************************************************
Mode

/home/bayes/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/bayes/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/bayes/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/bayes/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


          ----- Results -------              
                                             
metric_ac_score: 0.0233236151603
Precision: 0.333333333333
Accuracy:  0.0233236151603
F1: 0.0285204991087
Method Score: 0.0233236151603
Recall: 0.0148975791434
Model: LogisticRegression===>
--------------------------------
Data: data_2
--------------------------------
Method: kfold
--------------------------------
          ----- Results -------              
                                             
metric_ac_score: 0.0
Precision: 0.0
Accuracy:  0.0
F1: 0.0
Method Score: 0.0
Recall: 0.0
Model: LogisticRegression===>
--------------------------------
Data: data_3
--------------------------------
Method: kfold
--------------------------------
          ----- Results -------              
                                             
metric_ac_score: 0.148688046647
Precision: 0.508771929825
Accuracy:  0.148688046647
F1: 0.146402111032
Method Score: 0.148688046647
Recall: 0.0953127129037
Model: 

/home/bayes/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/bayes/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


          ----- Results -------              
                                             
metric_ac_score: 0.110787172012
Precision: 0.611111111111
Accuracy:  0.110787172012
F1: 0.11755674172
Method Score: 0.110787172012
Recall: 0.0712744697279
Model: RandomForestClassifier===>
--------------------------------
Data: data_2
--------------------------------
Method: kfold
--------------------------------


...

          ----- Results -------              
                                             
metric_ac_score: 0.122448979592
Precision: 0.604166666667
Accuracy:  0.122448979592
F1: 0.126513492838
Method Score: 0.122448979592
Recall: 0.0787232592996
Model: RandomForestClassifier===>
--------------------------------
Data: data_4
--------------------------------
Method: kfold
--------------------------------


/home/bayes/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/bayes/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


          ----- Results -------              
                                             
metric_ac_score: 0.069970845481
Precision: 0.607142857143
Accuracy:  0.069970845481
F1: 0.0781144781145
Method Score: 0.069970845481
Recall: 0.0448630603624
Model: RandomForestClassifier===>
--------------------------------
Data: data_5
--------------------------------
Method: kfold
--------------------------------


/home/bayes/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/bayes/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/bayes/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/bayes/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


          ----- Results -------              
                                             
metric_ac_score: 0.0728862973761
Precision: 0.41568627451
Accuracy:  0.0728862973761
F1: 0.080691909511
Method Score: 0.0728862973761
Recall: 0.0470659036199
Model: BernoulliNB===>
--------------------------------
Data: data_1
--------------------------------
Method: kfold
--------------------------------
          ----- Results -------              
                                             
metric_ac_score: 0.0204081632653
Precision: 0.291666666667
Accuracy:  0.0204081632653
F1: 0.0249554367201
Method Score: 0.0204081632653
Recall: 0.0130353817505
Model: BernoulliNB===>
--------------------------------
Data: data_2
--------------------------------
Method: kfold
--------------------------------
          ----- Results -------              
                                             
metric_ac_score: 0.0
Precision: 0.0
Accuracy:  0.0
F1: 0.0
Method Score: 0.0
Recall: 0.0
Model: BernoulliNB=